<a href="https://colab.research.google.com/github/Zeeshan0123/Testing_functions/blob/main/testing_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Testing different codes for testing**

In [ ]:
prompt = "Hello, can you assist me?"
prompt_lower = prompt.lower()  # Converts the prompt to 'hello, can you assist me?'

general_keywords = ["hi", "hello", "greetings", "how are you", "general knowledge", "common", "assist", "help", "thanks"]

result = any(keyword in prompt_lower for keyword in general_keywords)

In [ ]:
result # ok so it return a boolian value

True

In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 12.3 MB/s eta 0:00:00


In [ ]:
import torch
from sentence_transformers import util,SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
def answer_from_spreadsheet(question):
    # List of questions and answers
    qa_list = [
        {"Question": "What are the working hours?", "Answer": "The standard working hours are 9 AM to 6 PM."},
        {"Question": "What is the company policy on remote work?", "Answer": "Remote work is allowed based on manager approval."},
        {"Question": "How do I apply for leave?", "Answer": "You can apply for leave through the HR portal."},
        {"Question": "What is the company's dress code?", "Answer": "The dress code is business casual."},
        {"Question": "Who should I contact for IT support?", "Answer": "You should contact the IT support team at it-support@company.com."}
    ]

    # Convert the list of questions into a list of strings
    questions_list = [item['Question'] for item in qa_list]

    # Encode the input question and the list of questions
    question_embedding = model.encode(question, convert_to_tensor=True)
    questions_embeddings = model.encode(questions_list, convert_to_tensor=True)

    # Compute cosine similarity between input question and all questions in the list
    cosine_scores = util.pytorch_cos_sim(question_embedding, questions_embeddings)

    threshold = 0.8
    print("Cosine Score:",cosine_scores)
    print("torch.where:",torch.where(cosine_scores > threshold))
    above_threshold_indices = torch.where(cosine_scores > threshold)[1]

    print("Above Threshold Indices:",above_threshold_indices)
    if len(above_threshold_indices) > 0:
        suggested_questions = []
        for index in above_threshold_indices:
            suggested_question = qa_list[index.item()]['Question']
            suggested_answer = qa_list[index.item()]['Answer']
            similarity_score = cosine_scores[0, index].item()
            suggested_questions.append({
                'question': suggested_question,
                'answer': suggested_answer,
                'similarity': f"{similarity_score * 100:.2f}%"
            })
        response = {
            'status': 'success',
            'question': question,
            'suggested_questions': suggested_questions
        }
    else:
        response = {
            'status': 'fail',
            'message': 'No similar questions found above the similarity threshold.'
        }

    return response


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Sample questions to test
test_questions = [
    "What are the working hours?",
    "When does the office open and close?",
    "Can we work from home?",
    "What is the best coding language?",
    "H0w do 1 apply f0r le@ve?"
]

# Running the test
for question in test_questions:
    response = answer_from_spreadsheet(question)
    print(f"Input Question: {question}")
    print(f"Response: {response}")
    print('-' * 40)


Cosine Score: tensor([[1.0000, 0.3504, 0.1588, 0.2853, 0.0161]])
torch.where: (tensor([0]), tensor([0]))
Above Threshold Indices: tensor([0])
Input Question: What are the working hours?
Response: {'status': 'success', 'question': 'What are the working hours?', 'suggested_questions': [{'question': 'What are the working hours?', 'answer': 'The standard working hours are 9 AM to 6 PM.', 'similarity': '100.00%'}]}
----------------------------------------
Cosine Score: tensor([[0.3697, 0.1999, 0.1814, 0.2608, 0.0485]])
torch.where: (tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))
Above Threshold Indices: tensor([], dtype=torch.int64)
Input Question: When does the office open and close?
Response: {'status': 'fail', 'message': 'No similar questions found above the similarity threshold.'}
----------------------------------------
Cosine Score: tensor([[0.3496, 0.4574, 0.1737, 0.1363, 0.1202]])
torch.where: (tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))
Above Thresh

In [ ]:
def generate_AI_answer_Agent(model_id, prompt):
    feedback_manager = FeedbackManager('feedback_data.json')
    query_embedding = model.encode(prompt, convert_to_tensor=True)
    similar_feedback = feedback_manager.find_similar_query(model_id, query_embedding)

    if similar_feedback:
        return {"answer": similar_feedback['corrected_answer']}

    if not prompt:
        return {"error": "prompt is required"}, 400

    output = []
    is_general = is_general_question(prompt)
    chain = make_chain(model_id)

    if is_general:
        QUERY_PROMPT = "Answer me with general knowledge and ignore the context if query is general like greetings or general question"
        response = chain.invoke({"question": f"{QUERY_PROMPT}.\nUSER QUESTION : {prompt}", 'chat_history': ""})
    else:
        response = chain.invoke({"question": prompt, 'chat_history': ""})
        answer = response.get("answer", "No answer available")
        if "sorry" in answer.lower() or "no entiendo tu pregunta" in answer.lower():
            QUERY_PROMPT = "Answer me with general knowledge and ignore the context if query is general like greetings or general question"
            response = chain.invoke({"question": f"{QUERY_PROMPT}.\nUSER QUESTION : {prompt}", 'chat_history': ""})
    source = response.get("source_documents", [])
    output.append(response.get("answer", "No answer available"))
    doc_output = []
    if not is_general and "sorry" not in answer.lower():
        answer_embedding = model.encode(answer, convert_to_tensor=True)
        for document in source:
            if document.metadata['name'] != "APIResource":
                document_embedding = model.encode(document.page_content, convert_to_tensor=True)
                relevancy = util.pytorch_cos_sim(answer_embedding, document_embedding).item()
                if relevancy > 0.63:
                    if 'page_number' in document.metadata and 'link' in document.metadata:
                            page_no = document.metadata['page_number']
                            link = document.metadata['link']
                            api_endpoint = f"https://products-api.macaan.ai/v1/models/13301/assets"
                            headers = {"accept": "application/json"}
                            response = requests.get(api_endpoint, headers=headers)
                            if response.status_code == 200:
                                assets = response.json()
                                for asset in assets:
                                    if asset['description'] == link:
                                        document.metadata['link'] = asset['url'] + "#page=" + str(page_no)
                                    doc_output.append(document)
    output.append(doc_output)
    return {"answer": output[0], "source": [doc.metadata['link'] for doc in doc_output if 'link' in doc.metadata] }


In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os

from groq import Groq
from google.colab import userdata

client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models, which are also known as rapid language models or high-speed language models, have gained significant importance in recent years due to their ability to process and generate human-like language quickly and efficiently. Here are some reasons why fast language models are crucial:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation services to respond quickly to user queries, making them more interactive and user-friendly.
2. **Scalability**: As the volume of user-generated content and data continues to grow exponentially, fast language models can process and analyze large amounts of text data quickly, making them essential for scalable applications.
3. **Real-time Language Understanding**: Fast language models can quickly understand natural language text, enabling applications to respond to user queries without latency, making them more effective in situations where speed is

In [ ]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')

'import os\r\n\r\nfrom groq import Groq\r\n\r\nclient = Groq(\r\n    api_key=os.environ.get("GROQ_API_KEY"),\r\n)\r\n\r\nchat_completion = client.chat.completions.create(\r\n    messages=[\r\n        {\r\n            "role": "user",\r\n            "content": "Explain the importance of fast language models",\r\n        }\r\n    ],\r\n    model="llama3-8b-8192",\r\n)\r\n\r\nprint(chat_completion.choices[0].message.content)'

In [ ]:
pip install langchain langchain-community langchain-groq langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing

In [ ]:
pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.5/602.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata
userdata.get('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
llm.invoke("hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.001027061, 'queue_time': 0.01671279, 'total_time': 0.021860394}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-bcc1ead0-d645-4778-baab-5992a063eedc-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [ ]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

emb_model = "sentence-transformers/facebook-dpr-ctx_encoder-multiset-base"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
chain = ConversationalRetrievalChain.from_llm(llm,
                                              chain_type="stuff",
                                              retriever=retriever,
                                              return_source_documents=True,)


In [ ]:
chat_history = []
query = "Explain the API-Bank benchmark"
result = chain({"question": query, "chat_history": chat_history})


In [ ]:
result

{'question': 'Explain the API-Bank',
 'chat_history': [],
 'answer': 'API-Bank is a benchmark for evaluating the performance of tool-augmented Large Language Models (LLMs). It contains:\n\n1. 53 commonly used API tools\n2. A complete tool-augmented LLM workflow\n3. 264 annotated dialogues that involve 568 API calls\n\nThe selection of APIs is diverse, including various categories such as:\n\n* Search engines\n* Calculator\n* Calendar queries\n* Smart home control\n* Schedule management\n* Health data management\n* Account authentication workflow\n\nTo use an API, LLM first searches for the right API using an API search engine, and then uses the corresponding documentation to make a call.',
 'source_documents': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='API-Bank (Li et al. 2023) is a benchmark for evaluating the performance of tool-augmented LLMs. It contains 53 commonly used API tools, a complete tool-augmented LLM workflow, and

In [ ]:
answer = result.get("answer", "No answer available") # Both lines output is same
answer = result.get("answer") # Both lines output is same

In [ ]:
answer

'I apologize, but I don\'t have information on the "API-Bank benchmark". Could you please provide more context or details about what API-Bank benchmark is? Is it a specific benchmark for measuring the performance of artificial intelligence or machine learning models? Or is it a benchmark for a specific type of system or application?'

In [ ]:
source = result.get("source_documents", []) # Both lines output is same
source = result.get("source_documents") # Both lines output is same

In [ ]:
source

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='inquired about current trends in anticancer drug discovery;\nselected a target;\nrequested a scaffold targeting these compounds;\nOnce the compound was identified, the model attempted its synthesis.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='are imported by that file, and so on.\\nFollow a language and framework appropriate best practice file naming convention.\\nMake sure that files contain all imports, types etc. The code should be fully functional. Make sure that code in different files are compatible with each other.\\nBefore you finish, double check that all parts of the architecture is

In [ ]:
chat_history.append((query, result["answer"]))

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Question': [
        'What is AI?',
        'What is machine learning?',
        'What is deep learning?'
    ],
    'Answer': [
        'AI is artificial intelligence.',
        'Machine learning is a subset of AI.',
        'Deep learning is a subset of machine learning.'
    ]
})

In [ ]:
import torch
cosine_scores = torch.tensor([[0.75, 0.85, 0.60]])
threshold = 0.8
torch.where(cosine_scores > threshold)

(tensor([0]), tensor([1]))

In [ ]:
# pip install langchain-huggingface
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# emb_model = "sentence-transformers/facebook-dpr-ctx_encoder-multiset-base"
# model = HuggingFaceEmbeddings(model_name=emb_model)
from sentence_transformers import SentenceTransformer, util
question = ''
question_embedding = model.encode(question, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(question_embedding, questions_embeddings)

SyntaxError: invalid syntax (<ipython-input-13-d0ba60284a27>, line 4)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question = 'Briefly explain the deep learning concept?'
question_embedding = model.encode(question, convert_to_tensor=True)

In [ ]:
question_embedding

tensor([-7.2478e-02, -4.8343e-02,  3.2450e-02,  3.8402e-02,  1.1738e-02,
        -9.8999e-03, -7.1407e-03,  4.9000e-03, -1.1765e-02, -3.7682e-02,
        -8.0266e-02,  3.2115e-02, -2.4628e-02, -5.5504e-02, -9.3779e-02,
        -5.4799e-03,  4.5033e-02,  5.2643e-02, -1.4426e-01, -3.5482e-02,
         5.8412e-02, -1.4294e-02, -5.7079e-02,  8.4832e-03,  1.0424e-02,
         4.8283e-02,  3.1935e-02, -7.5325e-03,  5.4744e-02, -3.7134e-04,
         7.0979e-02,  1.9828e-02,  6.2964e-02,  9.6944e-02, -4.3333e-02,
         9.2797e-02, -7.4723e-02,  4.7650e-02, -2.2146e-02,  1.6618e-02,
        -2.3716e-02, -6.0485e-02, -4.5484e-03,  3.7265e-02,  1.0979e-01,
         2.2397e-02,  3.6423e-02, -6.0206e-02,  1.9359e-02, -2.3895e-02,
        -6.6098e-02, -6.5291e-03, -8.2879e-02,  8.4919e-02, -2.3188e-02,
         5.0021e-02,  3.8109e-02,  2.1977e-02, -6.6971e-02,  8.1494e-03,
         6.0626e-02, -1.0474e-01,  6.8492e-03,  2.5424e-02,  8.3855e-02,
        -6.0060e-02,  1.5842e-02, -1.4382e-03,  5.5

In [ ]:
questions_embeddings = model.encode(df['Question'].tolist(), convert_to_tensor=True)

In [ ]:
cosine_scores = util.pytorch_cos_sim(question_embedding, questions_embeddings)
cosine_scores

tensor([[0.4248, 0.5995, 0.8597]])

In [ ]:
import torch
threshold = 0.8
above_threshold_indices = torch.where(cosine_scores > threshold)[1]
above_threshold_indices

tensor([2])

In [ ]:
torch.where(cosine_scores > threshold)

(tensor([0]), tensor([2]))

In [ ]:
len(above_threshold_indices)

1